In [2]:
import torch
import torch.nn as nn
import math

from codespace.model.multihead_attention_transformer import _get_activation_fn
from mamba_ssm import Mamba


class FC_Decoder(nn.Module):
    def __init__(self, num_class, dim_feedforward, dropout, input_num=3):
        super().__init__()
        self.num_class = num_class

        self.output_layer1 = nn.Linear(
            dim_feedforward * input_num, dim_feedforward // input_num
        )
        self.activation1 = torch.nn.Sigmoid()
        self.dropout1 = torch.nn.Dropout(dropout)

        self.output_layer3 = nn.Linear(2 * (dim_feedforward // input_num), num_class)
        self.residue_attn = Mamba(d_model=480, d_state=16, d_conv=4, expand=2)
        self.residue_linear1 = nn.Linear(2000, dim_feedforward // input_num)
        self.output_layer2 = nn.Linear(2 * (dim_feedforward // input_num), num_class)

    def forward(self, hs, residue):  # hs[3, 32, 512] residue[32,2000,480]
        residue = self.residue_attn(residue)  # residue[32,2000,480]
        residue = nn.functional.adaptive_avg_pool1d(residue, output_size=1).squeeze(
            -1
        )  # residue[32,2000]
        residue = self.residue_linear1(residue)  # residue[32,512//3]

        # 维度转换 第0维和第1维互换
        hs = hs.permute(1, 0, 2)  # [32, 3, 512]
        # 按第一维度展开
        hs = hs.flatten(1)  # [32,512*3]

        hs = self.output_layer1(hs)  # [32,512//3]

        conca_hs_residue = torch.cat((hs, residue), dim=1)
        # sigmoid
        conca_hs_residue = self.activation1(conca_hs_residue)
        conca_hs_residue = self.dropout1(conca_hs_residue)
        # (512//3,GO标签数)
        out = self.output_layer3(conca_hs_residue)
        return out


if __name__ == "__main__":
    hs = torch.rand(3, 32, 512).to("cuda:0")
    residue = torch.rand(32, 2000, 480).to("cuda:0")
    model = FC_Decoder(45, 512, 0.1).to("cuda:0")
    out = model(hs, residue)
    print(out.shape)

ModuleNotFoundError: No module named 'codespace'

In [ ]:
def read_residue(usefor, aspect, model_name, organism_num):
    residue_name = f"{usefor}_residue_{aspect}.pkl"
    file_path = os.path.join(
        finetune_data_path, organism_num, f"residue_{model_name}", residue_name
    )

    residue = pd.read_pickle(file_path)
    # 找到张量的最小值和最大值
    residue_min = residue.min(dim=2, keepdim=True).values
    residue_max = residue.max(dim=2, keepdim=True).values

    # 执行 min-max 归一化
    residue = (residue - residue_min) / (residue_max - residue_min)
    return residue

In [7]:
import os
import pandas as pd
import torch

In [4]:
dataset_path_in_kioedru = "/home/kioedru/code/SSGO/data"
dataset_path_in_Kioedru = "/home/Kioedru/code/SSGO/data"

if os.path.exists(dataset_path_in_kioedru):
    dataset_path = dataset_path_in_kioedru
else:
    dataset_path = dataset_path_in_Kioedru

finetune_data_path = os.path.join(dataset_path, "finetune")

In [10]:
usefor = "train"
aspect = "P"
model_name = "esm2"
organism_num = "9606"
residue_name = f"{usefor}_residue_{aspect}.pkl"
file_path = os.path.join(
    finetune_data_path, organism_num, f"residue_{model_name}", residue_name
)

residue = pd.read_pickle(file_path)
# # 找到张量的最小值和最大值
# residue_min = residue.min(dim=2, keepdim=True).values
# print(residue_min)
# residue_max = residue.max(dim=2, keepdim=True).values

# # 执行 min-max 归一化
# residue = (residue - residue_min) / (residue_max - residue_min)
layernorm = torch.nn.LayerNorm(480)
residue = layernorm(residue)
print(torch.isnan(residue).any())

tensor(False)
